In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.h3 import (
    get_h3_cells_for_nodes,
    get_h3_cells_for_bbox,
    plot_h3_cells_on_folium,
)
from package.mcr5.h3_osm_interaction import get_location_mappings_for_cells
from package.mcr5.mcr5 import MCR5
from package.osm.osm import add_nearest_osm_node_id
from package.mcr5.labels import read_labels_for_nodes
from package.minute_city import minute_city

from package.geometa import GeoMeta
from package.mcr.data import NetworkType

In [4]:
geo_meta_path = "../data/geometa.pkl"
city_id = "Koeln"
stops = "../data/cleaned/stops.csv"
structs = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
geo_data = OSMData(
	geo_meta,
	city_id,
	additional_network_types=[NetworkType.DRIVING],
)
nodes = geo_data.osm_nodes
edges = geo_data.osm_edges
driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

[19:15:29] INFO     Using existing OSM data                                                              ]8;id=741833;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=547808;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=894048;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=873045;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[19:15:34] INFO     Using existing OSM data                                                              ]8;id=258601;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=775618;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (driving)                                              ]8;id=229977;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=631125;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

In [6]:
h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
driving_h3_cells = get_h3_cells_for_nodes(driving_nodes[["lat", "lon"]].to_dict("records"), 9)
h3_cells = h3_cells.intersection(driving_h3_cells)

In [7]:
location_mappings, invalid_h3_cells = get_location_mappings_for_cells(list(h3_cells), nodes, max_tries=20)
invalid_h3_cells

[]

In [8]:
car_location_mappings, invalid_h3_cells = get_location_mappings_for_cells(
    list(h3_cells), driving_nodes, max_tries=40
)

In [9]:
from command.step_config import (
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_bicycle_only_config,
	get_public_transport_only_config,
    get_walking_only_config,
)


In [10]:
configs = {}
initial_steps, repeating_steps = get_bicycle_public_transport_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    bicycle_price_function="next_bike_no_tariff",
    bicycle_location_path=bicycle_location_path,
    structs_path=structs,
    stops_path=stops,
)
configs["bicycle_public_transport"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}

initial_steps, repeating_steps = get_car_only_config(
    geo_meta_path,
    city_id,
)
configs["car"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": car_location_mappings,
    "max_transfers": 1,
}


initial_steps, repeating_steps = get_bicycle_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    bicycle_price_function="next_bike_no_tariff",
    bicycle_location_path=bicycle_location_path,
)
configs["bicycle"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}


initial_steps, repeating_steps = get_public_transport_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    structs_path=structs,
    stops_path=stops,
)
configs["public_transport"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}

initial_steps, repeating_steps = get_walking_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
)
configs["walking"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 0,
}


[19:15:36] INFO     Using existing OSM data                                                              ]8;id=886910;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=846250;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=863486;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=424822;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[19:15:41] INFO     Using existing OSM data                                                              ]8;id=624271;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=266582;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (cycling)                                              ]8;id=545067;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=877639;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[19:15:45] INFO     Fetching POI for runtime optimization                                         ]8;id=300199;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=348377;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py#72\72]8;;\

...

           INFO     Reading stops done (0.02 seconds)                                       ]8;id=274560;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=642447;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#141\141]8;;\

[19:16:20] INFO     Using existing OSM data                                                              ]8;id=674947;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=863462;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=580327;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=69691;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[19:16:25] INFO     Fetching POI for runtime optimization                                        ]8;id=26118;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=279814;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py#165\165]8;;\

           INFO     Fetching POI for runtime optimization done (0.01 seconds)                    ]8;id=174870;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=37355;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py#165\165]8;;\

In [11]:
for key, config in configs.items():
    mcr5 = MCR5(**config["init_kwargs"])

    rlog.info(f"Running MCR5 for {key}")
    print(f"Running MCR5 for {key}")

    mcr5_output_path = os.path.join("../data/mcr5/Koeln", key)

    location_mappings = config["location_mappings"]

    errors = mcr5.run(
        location_mappings, start_time="08:00:00", output_dir=mcr5_output_path, max_transfers=config["max_transfers"]
    )


[19:16:26] INFO     Running MCR5 for bicycle_public_transport                                        ]8;id=102983;file:///tmp/ipykernel_89635/494235893.py\494235893.py]8;;\:]8;id=512620;file:///tmp/ipykernel_89635/494235893.py#4\4]8;;\

Running MCR5 for bicycle_public_transport


Starting:   0%|          | 0/257 [00:00<?, ?it/s]

thread '<unnamed>' panicked at 'assertion failed: !bags.is_empty()', /home/moritz/dev/uni/mlc/src/mlc.rs:152:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at 'assertion failed: !bags.is_empty()', /home/moritz/dev/uni/mlc/src/mlc.rs:152:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at 'assertion failed: !bags.is_empty()', /home/moritz/dev/uni/mlc/src/mlc.rs:152:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at 'assertion failed: !bags.is_empty()', /home/moritz/dev/uni/mlc/src/mlc.rs:152:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at 'assertion failed: !bags.is_empty()', /home/moritz/dev/uni/mlc/src/mlc.rs:152:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at 'assertion failed: !bags

[19:16:54] INFO     All processes finished.                                                             ]8;id=967274;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=342217;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

...

[19:17:45] INFO     All processes finished.                                                             ]8;id=730454;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=151793;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

           INFO     Running MCR5 for walking                                                         ]8;id=110876;file:///tmp/ipykernel_89635/494235893.py\494235893.py]8;;\:]8;id=110360;file:///tmp/ipykernel_89635/494235893.py#4\4]8;;\

Running MCR5 for walking


Starting:   0%|          | 0/257 [00:00<?, ?it/s]

[19:17:55] INFO     All processes finished.                                                             ]8;id=448808;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=384950;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

In [13]:
import pickle
errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle/errors.pkl")
print(errors[0]["logs"])

2023-10-23 19:17:05,065 - mcr5-891fa1998afffff - DEBUG - Starting MCR with config: {'disable_paths': True, 'path_manager': None, 'output_format': <OutputFormat.DF_FEATHER: 'df_feather'>, 'logger': <Logger mcr5-891fa1998afffff (DEBUG)>, 'timer': <package.logger.Timer object at 0x7fe3a7f1fdc0>, 'enable_limit': True, 'initial_steps': [[WalkingStep]], 'repeating_steps': [[BicycleStep], [WalkingStep]]}
2023-10-23 19:17:05,065 - mcr5-891fa1998afffff - DEBUG - Starting MCR with config: {'disable_paths': True, 'path_manager': None, 'output_format': <OutputFormat.DF_FEATHER: 'df_feather'>, 'logger': <Logger mcr5-891fa1998afffff (DEBUG)>, 'timer': <package.logger.Timer object at 0x7fe3a7f1fdc0>, 'enable_limit': True, 'initial_steps': [[WalkingStep]], 'repeating_steps': [[BicycleStep], [WalkingStep]]}
2023-10-23 19:17:05,066 - mcr5-891fa1998afffff - INFO - Running initial step
2023-10-23 19:17:05,066 - mcr5-891fa1998afffff - INFO - Running initial step
2023-10-23 19:17:05,087 - mcr5-891fa1998afff

**small area**  
2s with limits  
20s without limits


**full area**  
2s with limits  
9m58s without limits